# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

weather_df = pd.read_csv("../WeatherPy/cities.csv")

weather_df.drop(columns =['Unnamed: 0'])
weather_df.head()

,Unnamed: 0,cities,lat,long,humidity,max temp,windspeed,cloudiness,date
0,0,Puerto Ayora,-0.7393,-90.3518,91,77.00,1.99,4,1970-01-01 00:00:01.615603712
1,1,Provideniya,64.3833,-173.3000,96,30.45,31.29,100,1970-01-01 00:00:01.615603713
2,2,Hof,50.3167,11.9167,96,36.00,11.99,100,1970-01-01 00:00:01.615603714
3,3,Faanui,-16.4833,-151.7500,63,83.35,1.63,84,1970-01-01 00:00:01.615603715
4,4,Deputatsky,69.3000,139.9000,80,-11.34,1.48,19,1970-01-01 00:00:01.615603717


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["lat", "long"]]
humidity = weather_df["humidity"]

In [4]:
# Plot Heatmap
gmaps.configure(api_key = g_key)
fig = gmaps.figure(zoom_level = 2, center = (42.6001, -73.9662))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
perfect_weather_df = weather_df.loc [ (weather_df ["windspeed"] <10) & (weather_df ["cloudiness"] == 0) & (weather_df ["max temp"] > 70) 
                                    & (weather_df ["max temp"] < 80)]
del perfect_weather_df["Unnamed: 0"]
perfect_weather_df.head()


,cities,lat,long,humidity,max temp,windspeed,cloudiness,date
10,Malakal,9.5334,31.6605,22,79.88,3.11,0,1970-01-01 00:00:01.615603724
16,Atuona,-9.8000,-139.0333,58,79.86,1.48,0,1970-01-01 00:00:01.615603733
24,Belmonte,-15.8631,-38.8828,65,77.00,8.05,0,1970-01-01 00:00:01.615603744
127,Cidreira,-30.1811,-50.2056,77,70.88,5.66,0,1970-01-01 00:00:01.615603884
163,Caravelas,-17.7125,-39.2481,81,72.75,5.19,0,1970-01-01 00:00:01.615603932


In [17]:
len(perfect_weather_df)

17

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#create hotel_df data frame from cleaned data above

hotel_df = perfect_weather_df

hotel_df["Hotel Name"] = ""

hotel_df.head()

<ipython-input-18-2789f4903ea1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,cities,lat,long,humidity,max temp,windspeed,cloudiness,date,Hotel Name
10,Malakal,9.5334,31.6605,22,79.88,3.11,0,1970-01-01 00:00:01.615603724,
16,Atuona,-9.8000,-139.0333,58,79.86,1.48,0,1970-01-01 00:00:01.615603733,
24,Belmonte,-15.8631,-38.8828,65,77.00,8.05,0,1970-01-01 00:00:01.615603744,
127,Cidreira,-30.1811,-50.2056,77,70.88,5.66,0,1970-01-01 00:00:01.615603884,
163,Caravelas,-17.7125,-39.2481,81,72.75,5.19,0,1970-01-01 00:00:01.615603932,


In [19]:
# Store latitude and longitude in locations
locations = hotel_df[["lat", "long"]]
humidity = hotel_df["humidity"]

# Plot Heatmap
gmaps.configure(api_key = g_key)
fig = gmaps.figure(zoom_level = 2, center = (42.6001, -73.9662))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Loop through the hotel dataframe and find nearest hotel
for index, row in hotel_df.iterrows():

    lat = row["lat"]
    lng = row["long"]
    coords = f"{lat},{lng}"

    params = {
        "location": coords, 
        "types": "hotel",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_data = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "no hotel found"

hotel_df

/Users/summyabakshi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,cities,lat,long,humidity,max temp,windspeed,cloudiness,date,Hotel Name
10,Malakal,9.5334,31.6605,22,79.88,3.11,0,1970-01-01 00:00:01.615603724,Malakal
16,Atuona,-9.8000,-139.0333,58,79.86,1.48,0,1970-01-01 00:00:01.615603733,Atuona
24,Belmonte,-15.8631,-38.8828,65,77.00,8.05,0,1970-01-01 00:00:01.615603744,Belmonte
127,Cidreira,-30.1811,-50.2056,77,70.88,5.66,0,1970-01-01 00:00:01.615603884,CIDREIRA
163,Caravelas,-17.7125,-39.2481,81,72.75,5.19,0,1970-01-01 00:00:01.615603932,Caravelas
166,Falam,22.9167,93.6833,19,79.45,1.28,0,1970-01-01 00:00:01.615603936,Falam
234,Vila Velha,-20.3297,-40.2925,69,77.00,5.75,0,1970-01-01 00:00:01.615604026,Vila Velha
254,Hualmay,-11.0964,-77.6139,84,71.67,7.49,0,1970-01-01 00:00:01.615604053,Huacho
257,Yanam,16.7333,82.2167,78,78.80,2.71,0,1970-01-01 00:00:01.615604057,Yanam
289,Les Cayes,18.2000,-73.7500,82,72.75,6.62,0,1970-01-01 00:00:01.615604102,Les Cayes


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
